Primary Objective: complete all the steps below and return the risk analysis of your seven (7) stock portfolio
against the S&P500 (SPY), Russell 2000 (IWM), and the Dow Jones Industrial Average (DIA).

Requirements:
3. Create a correlation matrix showing the correlations between the equal-weighted portfolio, 3 ETFs, and your 7 stocks.
4. Output these two tables and the matrix into a PDF file. You must submit both your Python code file and your PDF output file.


In [1]:
#For package verification
#pip install yfinance
#7 stocks picked are MSFT, AAPL, GOOG, AMD, BRK.A, JPM, and DAL
#This part was completed thanks to the help of following documentations:
#https://algotrading101.com/learn/yfinance-guide/
#https://docs.python.org/3/library/datetime.html
#https://stackoverflow.com/questions/2394235/detecting-a-us-holiday
#https://numpy.org/doc/stable/reference/generated/numpy.cov.html
#3. Covariance function - what's the catch?
#4. Tracking Errors? of what?


import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
import holidays
us_holidays = holidays.US()

In [51]:
#Create some useful function for calculation needs
def daily_return(stock,period):
    #Return a list of daily returns of a certain stock
    prices = yf.download(stock,period = period)
    return prices['Adj Close'].pct_change()

def volatility(stock, period):
    #Calculate market volatility based on the start and end date
    re = daily_return(stock,period)
    sd_return = np.std(re)
    return sd_return*np.sqrt(len(re))

def calculate_beta(stock,benchmark,period):
    #calculate covariance using stock and index info
    stock_return = daily_return(stock,period)[1:]
    index_return = daily_return(benchmark,period)[1:]
    #I couldn't figure out why the covariance function does not work for the life of me, so I took it to my hands
    i = np.array(index_return)
    s = np.array(stock_return)
    ai = np.mean(i)
    sa = np.mean(s)
    i1 = i-ai #benchmark daily return - average return
    s1 = s-sa #stock daily return - average return 
    r = np.dot(i1,s1) #take the dot product of them to calculate covariance
    cov = r/len(i) #divide by n - 1
    sd_i = np.std(i)**2 #get the index variance
    #Ideally Covariance should form something like this from pythonm, but np.cov doesn't seem to work
    #           stock    benchmark
    # stock    stock_var  cov
    #benchmark cov      benchmark_var
    #and beta is calculated using covariance of stock and benchmark/variance of stock
    return cov/sd_i

def total_return(stock,period):
    #Calculate total return over a certian period
    prices = yf.download(stock,period = period)
    return (prices['Adj Close'][-1] - prices['Adj Close'][0])/prices['Adj Close'][0]
    
def drawdown(stock, period, mode):
    prices = daily_return(stock,period)
    drawdowns = prices.rolling(5).max() - prices.rolling(5).min()
    if mode == 'a':
        return drawdowns.mean()
    elif mode == 'm':
        return drawdowns.max()
    return np.nan()

In [56]:
#Initiate analysis
risk_analysis = pd.DataFrame(columns=['Ticker','Portfolio Weight','Annualized Volatility(Trailing 3-month)',
                                      'Beta vs SPY','Beta vs IWM', 'Beta vs DIA', 
                                      'Average Weekly Drawdown','Maximum Weekly Drawdown',
                                      'Total Return over 10 years (in %)','Annualized Total Return over 10 years (in %)'])
risk_analysis['Ticker'] = ['MSFT', 'AAPL', 'GOOG', 'AMD', 'BLK', 'JPM', 'DAL']

for x in range(6):
    risk_analysis['Portfolio Weight'][x] = 1/len(risk_analysis['Ticker'])

#For task #2
market_table = pd.DataFrame(columns = ['Ticker', 'Corr', 'Cov', 'Tracking Errors', 'Sharpe Ratio','Volatility'])
market_tabl['Ticker'] = ['Portfolio','IWM','DIA','SPY']

In [57]:
#Make the table to risk_analysis
count = 0
for Ticker in risk_analysis['Ticker']:
    risk_analysis['Annualized Volatility(Trailing 3-month)'][count] = volatility(Ticker,'3mo')
    #Beta is measured against a benchmark.
    risk_analysis['Beta vs SPY'][count] = calculate_beta(Ticker,'SPY','1y') 
    risk_analysis['Beta vs IWM'][count] = calculate_beta(Ticker,'IWM','1y') 
    risk_analysis['Beta vs DIA'][count] = calculate_beta(Ticker,'DIA','1y')
    
    #Find average drawdown and weekly drawdown
    risk_analysis['Average Weekly Drawdown'][count] = drawdown(Ticker, '1y','a')
    risk_analysis['Maximum Weekly Drawdown'][count] = drawdown(Ticker, '1y','m')
    
    #Find returns
    x = total_return(Ticker,'10y')
    risk_analysis['Total Return over 10 years (in %)'][count] = x*100
    risk_analysis['Annualized Total Return over 10 years (in %)'][count] = ((1+x)**(1/10)- 1)*100
    count+=1

#Make the table for ETFs 
    

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [58]:
risk_analysis

,Ticker,Portfolio Weight,Annualized Volatility(Trailing 3-month),Beta vs SPY,Beta vs IWM,Beta vs DIA,Average Weekly Drawdown,Maximum Weekly Drawdown,Total Return over 10 years (in %),Annualized Total Return over 10 years (in %)
0,MSFT,0.142857,0.1821,1.206691,0.76235,1.274453,0.041389,0.089921,1004.456408,27.149559
1,AAPL,0.142857,0.203355,1.257334,0.836188,1.363741,0.042443,0.096724,740.62005,23.725716
2,GOOG,0.142857,0.208643,1.282543,0.842207,1.359309,0.045554,0.110057,311.817686,15.20478
3,AMD,0.142857,0.331715,2.114172,1.620009,2.150486,0.083212,0.186769,2082.650596,36.112629
4,BLK,0.142857,0.190924,1.168208,0.844556,1.3857,0.040273,0.101379,393.392948,17.305748
5,JPM,0.142857,0.160153,0.874705,0.648775,1.183925,0.037571,0.080772,340.321801,15.978375
6,DAL,NaN,0.265372,1.390863,1.264329,1.756228,0.065516,0.177079,269.618815,13.966027


In [60]:
market_table = pd.DataFrame(columns = ['Ticker', 'Corr', 'Cov', 'Tracking Errors', 'Sharpe Ratio','Volatility'])
market_table['Ticker'] = ['IWM','DIA','SPY']

In [64]:
returns = pd.DataFrame(index=['IWM','DIA','SPY'],columns = ['10 year return'])
for count in returns.index:
    returns['10 year return'][count] = total_return(count,'10y') 

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [65]:
returns

,10 year return
IWM,1.690252
DIA,2.136426
SPY,2.567402


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [141]:

r

0.00776421711196627

1.8872586057514427

1.8872586057514424